In [1]:
%load_ext bigdata
%hive_start
%timeout 300

In [2]:
!hdfs dfs -mkdir /tmp/drivers

mkdir: `/tmp/drivers': File exists


In [3]:
!hdfs dfs -ls /tmp/drivers/*

ls: `/tmp/drivers/*': No such file or directory


In [4]:
!hdfs dfs -tail /tmp/drivers/tbl0.csv
!hdfs dfs -tail /tmp/drivers/tbl1.csv

tail: `/tmp/drivers/tbl0.csv': No such file or directory
tail: `/tmp/drivers/tbl1.csv': No such file or directory


In [1]:
%%hive
DROP TABLE IF EXISTS tbl0;
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;
--
DROP TABLE IF EXISTS tbl1;
CREATE TABLE tbl1 (
    c1 INT,
    c2 INT,
    c3 STRING,
    c4 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'tbl1.csv' INTO TABLE tbl1; 

UsageError: Cell magic `%%hive` not found.


In [2]:
%%hive
DROP TABLE IF EXISTS t0;
CREATE TABLE t0 (
    c1 STRING,
    c2 ARRAY<CHAR(1)>, 
    c3 MAP<STRING, INT>
    )
    ROW FORMAT DELIMITED 
        FIELDS TERMINATED BY '\t'
        COLLECTION ITEMS TERMINATED BY ','
        MAP KEYS TERMINATED BY '#'
        LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'data.tsv' INTO TABLE t0;

DROP TABLE IF EXISTS t0;
OK
Time taken: 11.425 seconds
CREATE TABLE t0 (
    c1 STRING,
    c2 ARRAY<CHAR(1)>, 
    c3 MAP<STRING, INT>
    )
    ROW FORMAT DELIMITED 
        FIELDS TERMINATED BY '\t'
        COLLECTION ITEMS TERMINATED BY ','
        MAP KEYS TERMINATED BY '#'
        LINES TERMINATED BY '\n';
OK
Time taken: 0.795 seconds
LOAD DATA LOCAL INPATH 'data.tsv' INTO TABLE t0;
Loading data to table default.t0
OK
Time taken: 3.222 seconds


In [3]:
%%hive
--SELECT * FROM tbl0;
--SELECT * FROM tbl1;
SELECT * FROM t0;

--SELECT * FROM tbl0;
--SELECT * FROM tbl1;
SELECT * FROM t0;
OK
E	["b","g","f"]	{"jjj":3,"bbb":0,"ddd":9,"ggg":8,"hhh":2}
A	["a","f","c"]	{"ccc":2,"ddd":0,"aaa":3,"hhh":9}
B	["f","e","a","c"]	{"ddd":2,"ggg":5,"ccc":6,"jjj":1}
A	["a","b"]	{"hhh":9,"iii":5,"eee":7,"bbb":1}
C	["f","g","d","a"]	{"iii":6,"ddd":5,"eee":4,"jjj":3}
A	["c","d"]	{"bbb":2,"hhh":0,"ccc":4,"fff":1,"aaa":7}
A	["g","d","a"]	{"aaa":5,"fff":8,"ddd":2,"iii":0,"jjj":7,"ccc":1}
B	["b","a"]	{"fff":3,"hhh":1,"ddd":2}
E	["d","e","a","f"]	{"eee":4,"ccc":5,"iii":9,"fff":7,"ggg":6,"bbb":0}
B	["d","b","g","f"]	{"bbb":7,"jjj":9,"fff":5,"iii":4,"ggg":2,"eee":3}
C	["d","c","f","b"]	{"hhh":6,"eee":4,"iii":0,"fff":2,"jjj":1}
C	["d","e","a","c"]	{"bbb":7,"iii":6,"ggg":9}
D	["g","e","f","b"]	{"bbb":9,"aaa":3,"ccc":6,"fff":4,"eee":2}
E	["c","f"]	{"aaa":8,"ddd":5,"jjj":1}
B	["d","b"]	{"ccc":0,"jjj":6,"fff":7,"ddd":3,"aaa":2}
D	["f","e"]	{"ccc":0,"eee":6,"bbb":9,"ddd":3}
E	["e","b","f"]	{"bbb":6,"iii":3,"hhh":5,"fff":4,"ggg":9,"ddd":2}
D

In [9]:
%%hive
INSERT OVERWRITE LOCAL DIRECTORY 'output'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
--select tf1.*
select tf1.key, COUNT(tf1.key)
from t0
lateral view explode(c3) tf1
GROUP BY tf1.key;

--INSERT OVERWRITE LOCAL DIRECTORY 'output'
--ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
--select tf1.*
select tf1.key, COUNT(tf1.key)
from t0
lateral view explode(c3) tf1
GROUP BY tf1.key;
Query ID = root_20200209024052_6642f7fb-62e2-4090-87df-ae0be760f976
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1581215543186_0005, Tracking URL = http://12ea57c3bcba:8088/proxy/application_1581215543186_0005/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1581215543186_0005
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2020-02-09 02:41:02,636 Stage-1 map = 0%,  reduce = 0%
2020-02-09 

In [ ]:
%%hive
--INSERT OVERWRITE LOCAL DIRECTORY 'output'
--ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
--SELECT t0.c3, COUNT(t0.c3)
--FROM t0
--WHERE (t0.c3['jjj']=0)
--GROUP BY t0.c3;

SELECT
        category
    FROM (
        SELECT
            c3
        FROM
            t0
        LATERAL VIEW
            explode(c3) t0 AS category
    ) t1
    --LATERAL VIEW
        --explode(ranking) t0
;

In [160]:
%%hive
SELECT  fec, word ,count(*) FROM (
SELECT substr (c4,1,4) AS fec , word
FROM tbl0 lateral view explode(split(c4, '\\:')) temp_table AS word
) x
GROUP BY  fec ,word;

ON
    (t1.c1 = t2.c1  );
Query ID = root_20200208191822_950b3a85-d81c-4afd-b7ea-d3c5d11f7ef1
Total jobs = 1
SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/local/hive/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/local/hadoop/share/hadoop/common/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
2020-02-08 19:18:32	Starting to launch local task to process map join;	maximum memory = 477626368
2020-02-08 19:18:35	Dump the side-table for tag: 1 with group count: 10 into file: file:/tmp/root/51b149e5-d692-4c3a-8ded-97305de30c09/hive_2020-02-08_19-18-22_149_4478622126870134815-1/-local-10004/HashTable-Stage-3/MapJoin-mapfile291--.hashtable
2020-02-08 19:18:35	Uploaded 1 File to: file:/tmp/root/51b149e5-d69